<div style="background-color: salmon; padding: 60px;">
    <h1><b>Data Wrangling
</b></h1>
</div>

In [1]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd
from env import user, password, host
#from pyspark.sql.functions import * # for when you don't want to use F

In [2]:
# Generate spark object to initialize a local Spark JVM process
spark = SparkSession.builder.getOrCreate()

23/10/30 02:06:48 WARN Utils: Your hostname, Aswathys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.28 instead (on interface en0)
23/10/30 02:06:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/30 02:06:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/30 02:06:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Reading Data

In [3]:
def get_db_url(db):
    '''input df and output sql connection string'''
    return (f'mysql+pymysql://{user}:{password}@{host}/{db}')

## Data Acquisition

These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

These exercises use the cases, dept, and source tables from the 311_data on the Codeup MySQL server.

# 1. Read the cases, department, and source data into their own Spark dataframes.



# Cases

In [5]:
#sql query
url = get_db_url('311_data')
query = 'select source_id, source_username from source'

In [6]:
#make pandas df
pandas_df = pd.read_sql(query, url)
pandas_df.head()

,source_id,source_username
0,100137,Merlene Blodgett
1,103582,Carmen Cura
2,106463,Richard Sanchez
3,119403,Betty De Hoyos
4,119555,Socorro Quiara


In [7]:
# transform sources into a spark df:
sources = spark.createDataFrame(pandas_df)

In [9]:
query = 'SELECT * FROM dept'
depts = spark.createDataFrame(
pd.read_sql(query, url))

### load cases from 311_data from sql

In [10]:
#sql query
query = 'select * from cases limit 100000'

In [11]:
#pandas df
pandas_df = pd.read_sql(query, url)

In [12]:
#spark df
df = spark.createDataFrame(pandas_df)
df

DataFrame[case_id: bigint, case_opened_date: string, case_closed_date: string, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: bigint]

In [13]:
depts = spark.createDataFrame(pd.read_sql('SELECT * FROM dept', url))

In [14]:
#df.write.format()
df.write.parquet('data/cases_pqt', mode='overwrite')
sources.write.parquet('data/sources_pqt', mode='overwrite')
depts.write.parquet('data/depts_pqt', mode='overwrite')

23/10/30 02:07:31 WARN TaskSetManager: Stage 0 contains a task of very large size (1472 KiB). The maximum recommended task size is 1000 KiB.
23/10/30 02:07:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/10/30 02:07:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/10/30 02:07:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/10/30 02:07:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/10/30 02:07:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/10/30 02:07:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,

In [15]:
case = spark.read.parquet("data/cases_pqt")
dept = spark.read.parquet("data/depts_pqt")
source = spark.read.parquet("data/sources_pqt")

In [16]:
case.show(2,vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014210978           
 case_opened_date     | 2/1/18 11:25         
 case_closed_date     | 2/5/18 18:00         
 SLA_due_date         | 4/5/18 11:25         
 case_late            | NO                   
 num_days_late        | -58.72600694         
 case_closed          | YES                  
 dept_division        | Code Enforcement     
 service_request_type | Vacant Lot-Privat... 
 SLA_days             | 63.0                 
 case_status          | Closed               
 source_id            | CRM_Listener         
 request_address      | 300  SAN SALVADOR... 
 council_district     | 2                    
-RECORD 1------------------------------------
 case_id              | 1014210980           
 case_opened_date     | 2/1/18 11:26         
 case_closed_date     | 2/8/18 11:04         
 SLA_due_date         | 4/5/18 11:26         
 case_late            | NO                   
 num_days_late        | -56.015081

In [17]:
case.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'SLA_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district']

In [18]:
sources.show(5, vertical=True, truncate=False)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
-RECORD 1---------------------------
 source_id       | 103582           
 source_username | Carmen Cura      
-RECORD 2---------------------------
 source_id       | 106463           
 source_username | Richard Sanchez  
-RECORD 3---------------------------
 source_id       | 119403           
 source_username | Betty De Hoyos   
-RECORD 4---------------------------
 source_id       | 119555           
 source_username | Socorro Quiara   
only showing top 5 rows



# 2. Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json
  format, store these as `sources_csv` and `sources_json`
- Inspect your folder structure. What do you notice?

<div class="alert alert-info" role="alert">

During the write operation, Spark writes the data files and their corresponding CRC files. 

The ".crc" files (Cyclic Redundancy Check), which have the ".crc" extension, are checksum files associated with the data files. They store checksum values computed for each corresponding data file. 
- These checksums are used by Spark to validate the integrity of the data during read operations and are not typically meant to be accessed or processed directly by users. 
    
Once all the data files and CRC files are successfully written, Spark creates the "_SUCCESS" file. This file is created as an indicator that the write operation completed successfully and that all the data files, including their associated CRC files, are valid.

</div>

# 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [19]:
case.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

In [20]:
case.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: long (nullable = true)



In [22]:
#rename col
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

In [23]:
#convert data types

case = (
    case.withColumn("case_late", F.col("case_late")=="YES")
    .withColumn("case_closed", F.col("case_closed")=="YES")
)

case = case.withColumn("council_district", F.format_string("%03d", F.col("council_district")))

In [24]:
fmt = "M/d/yy H:mm" #saved to the underlying stored data

case = (
    case.withColumn('case_opened_date', F.to_timestamp(F.col('case_opened_date'), fmt))
    .withColumn('case_closed_date', F.to_timestamp(F.col('case_closed_date'), fmt))
    .withColumn('case_due_date', F.to_timestamp(F.col('case_due_date'), fmt))
)

In [25]:
# Cleanup text data
case = case.withColumn('request_address', F.lower(F.trim(F.col('request_address'))))

In [26]:
case.select("case_closed_date").sort("case_closed_date", ascending=False).first()

Row(case_closed_date=datetime.datetime(2018, 8, 7, 16, 42))

In [27]:
case.agg(F.max(F.col('case_closed_date'))).show()

+---------------------+
|max(case_closed_date)|
+---------------------+
|  2018-08-07 16:42:00|
+---------------------+



In [29]:
case.select(
    F.datediff(F.to_timestamp(F.lit("2018-08-07 16:42:00")), F.col("case_opened_date"))
).show(3)

+-------------------------------------------------------------+
|datediff(to_timestamp(2018-08-07 16:42:00), case_opened_date)|
+-------------------------------------------------------------+
|                                                          187|
|                                                          187|
|                                                          187|
+-------------------------------------------------------------+
only showing top 3 rows



In [30]:
case.select(
    F.datediff(F.to_timestamp(F.current_timestamp()), case.case_opened_date)
).show(3)

+-------------------------------------------------------------+
|datediff(to_timestamp(current_timestamp()), case_opened_date)|
+-------------------------------------------------------------+
|                                                         2097|
|                                                         2097|
|                                                         2097|
+-------------------------------------------------------------+
only showing top 3 rows



In [31]:
# Extract zipcode
case = case.withColumn('zipcode', F.regexp_extract(F.col('request_address'), r'\d+$', 0))

# Create a `case_lifetime` feature
case = (
    case.withColumn('case_age', F.datediff(F.to_timestamp(F.lit('2018-08-08 10:38:00')), 'case_opened_date'))
    .withColumn('days_to_closed', F.datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', F.when(
        F.col('case_closed'), F.col('days_to_closed')).otherwise(F.col('case_age')))
    .withColumn('curr_case_age', F.datediff(F.current_timestamp(), 'case_opened_date'))
    .drop('case_age', 'days_to_closed')
)

In [32]:
case.show(2, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 case_id              | 1014210978                             
 case_opened_date     | 2018-02-01 11:25:00                    
 case_closed_date     | 2018-02-05 18:00:00                    
 case_due_date        | 2018-04-05 11:25:00                    
 case_late            | false                                  
 num_days_late        | -58.72600694                           
 case_closed          | true                                   
 dept_division        | Code Enforcement                       
 service_request_type | Vacant Lot-Private (12")               
 SLA_days             | 63.0                                   
 case_status          | Closed                                 
 source_id            | CRM_Listener                           
 request_address      | 300  san salvador, san antonio, 78210  
 council_district     | 002                                    
 zipcode              | 78210           

## Data Exploratory Questions

### 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [33]:
case.filter("! case_closed")\
    .select('case_id', 'case_lifetime', 'case_opened_date', 'case_closed_date', 'num_days_late')\
    .sort(F.desc("num_days_late")).show(5)

+----------+-------------+-------------------+----------------+------------------+
|   case_id|case_lifetime|   case_opened_date|case_closed_date|     num_days_late|
+----------+-------------+-------------------+----------------+------------------+
|1014206419|          190|2018-01-30 16:54:00|            NULL|               NaN|
|1014263058|          168|2018-02-21 14:21:00|            NULL|               NaN|
|1014128388|          218|2018-01-02 09:39:00|            NULL|       211.5974884|
|1014130342|          218|2018-01-02 15:22:00|            NULL|211.35949069999995|
|1014131356|          217|2018-01-03 08:04:00|            NULL|       210.6458333|
+----------+-------------+-------------------+----------------+------------------+
only showing top 5 rows



In [34]:
case.filter("! case_closed")\
    .select('case_id', 'case_lifetime', 'curr_case_age', 'case_opened_date', 'case_closed_date', 'num_days_late')\
    .sort(F.desc("num_days_late")).show(5)

+----------+-------------+-------------+-------------------+----------------+------------------+
|   case_id|case_lifetime|curr_case_age|   case_opened_date|case_closed_date|     num_days_late|
+----------+-------------+-------------+-------------------+----------------+------------------+
|1014263058|          168|         2077|2018-02-21 14:21:00|            NULL|               NaN|
|1014206419|          190|         2099|2018-01-30 16:54:00|            NULL|               NaN|
|1014128388|          218|         2127|2018-01-02 09:39:00|            NULL|       211.5974884|
|1014130342|          218|         2127|2018-01-02 15:22:00|            NULL|211.35949069999995|
|1014131356|          217|         2126|2018-01-03 08:04:00|            NULL|       210.6458333|
+----------+-------------+-------------+-------------------+----------------+------------------+
only showing top 5 rows



### 2. How many Stray Animal cases are there?

In [35]:
case.groupby("service_request_type").count().sort(F.desc("count")).show(truncate=False)

+---------------------------------+-----+
|service_request_type             |count|
+---------------------------------+-----+
|No Pickup                        |8932 |
|Overgrown Yard/Trash             |6853 |
|Front Or Side Yard Parking       |5338 |
|Bandit Signs                     |4781 |
|Aggressive Animal(Non-Critical)  |3541 |
|Stray Animal                     |3497 |
|Damaged Cart                     |3465 |
|Cart Exchange Request            |3282 |
|Right Of Way/Sidewalk Obstruction|3121 |
|Junk Vehicle On Private Property |2897 |
|Pot Hole Repair                  |2802 |
|Lost/Stolen Cart                 |1990 |
|Cart Delivery                    |1987 |
|Animal Neglect                   |1928 |
|Dead Animal - Dog                |1701 |
|Alley-Way Maintenance            |1640 |
|Dead Animal - Cat                |1631 |
|Public Nuisance(Own Animal)      |1531 |
|Dead Animal - Misc               |1396 |
|Trapped/Confined Animal          |1299 |
+---------------------------------

In [36]:
case.filter(case.service_request_type == F.lit("Stray Animal")).count()

3497

### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [37]:
case.filter(case.dept_division == "Field Operations").filter(case.service_request_type != "Officer Standby").count()

15094

### 4.Convert the council_district column to a string column.



In [38]:
case = case.withColumn('council_district', F.col('council_district').cast('string'))

### 5. Extract the year from the case_closed_date column.


In [39]:
case.withColumn("year", F.year("case_closed_date")).select("service_request_type", 'case_closed_date', 'year')\
.show(5)

+--------------------+-------------------+----+
|service_request_type|   case_closed_date|year|
+--------------------+-------------------+----+
|Vacant Lot-Privat...|2018-02-05 18:00:00|2018|
|Junk Vehicle On P...|2018-02-08 11:04:00|2018|
|Minimum Housing-O...|2018-02-08 15:25:00|2018|
|Zoning Business i...|2018-02-05 09:22:00|2018|
|         Cart PickUp|2018-02-03 12:59:00|2018|
+--------------------+-------------------+----+
only showing top 5 rows



### 6.Convert num_days_late from days to hours in a new column num_hours_late.



In [40]:
case.withColumn("num_hours_late", F.round(F.expr("num_days_late * 24"), 1))\
.select("num_days_late", 'num_hours_late').show(5)

+-------------+--------------+
|num_days_late|num_hours_late|
+-------------+--------------+
| -58.72600694|       -1409.4|
| -56.01508102|       -1344.4|
| -122.8337153|       -2948.0|
| -126.0867361|       -3026.1|
| -4.936226852|        -118.5|
+-------------+--------------+
only showing top 5 rows



### 7.Join the cases data with the source and department data.



In [41]:
df = case.join(source, "source_id", "left").drop(source.source_id)

df.show(2, vertical=True)

-RECORD 0------------------------------------
 source_id            | CRM_Listener         
 case_id              | 1014210978           
 case_opened_date     | 2018-02-01 11:25:00  
 case_closed_date     | 2018-02-05 18:00:00  
 case_due_date        | 2018-04-05 11:25:00  
 case_late            | false                
 num_days_late        | -58.72600694         
 case_closed          | true                 
 dept_division        | Code Enforcement     
 service_request_type | Vacant Lot-Privat... 
 SLA_days             | 63.0                 
 case_status          | Closed               
 request_address      | 300  san salvador... 
 council_district     | 002                  
 zipcode              | 78210                
 case_lifetime        | 4                    
 curr_case_age        | 2097                 
 source_username      | CRM_Listener         
-RECORD 1------------------------------------
 source_id            | 120752               
 case_id              | 1014210980

In [42]:
df = df.join(dept, "dept_division", "left")\
    .drop(dept.dept_division)\
    .drop(dept.dept_name)\
    .withColumnRenamed('standardized_dept_name', 'department')\
    .withColumn('dept_subject_to_SLA', F.col('dept_subject_to_SLA') == 'YES')
df.show(2, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Code Enforcement     
 source_id            | CRM_Listener         
 case_id              | 1014210978           
 case_opened_date     | 2018-02-01 11:25:00  
 case_closed_date     | 2018-02-05 18:00:00  
 case_due_date        | 2018-04-05 11:25:00  
 case_late            | false                
 num_days_late        | -58.72600694         
 case_closed          | true                 
 service_request_type | Vacant Lot-Privat... 
 SLA_days             | 63.0                 
 case_status          | Closed               
 request_address      | 300  san salvador... 
 council_district     | 002                  
 zipcode              | 78210                
 case_lifetime        | 4                    
 curr_case_age        | 2097                 
 source_username      | CRM_Listener         
 department           | DSD/Code Enforcement 
 dept_subject_to_SLA  | true                 
-RECORD 1-------------------------

### 8.Are there any cases that do not have a request source?



In [43]:
df.filter(df.source_id == "null").count()

0

In [44]:
df.filter(df.source_id == "").count()

0

### 9. What are the top 10 service request types in terms of number of requests?



In [45]:
df.groupby("service_request_type").count().sort(F.desc("count")).show(10, 50)

+---------------------------------+-----+
|             service_request_type|count|
+---------------------------------+-----+
|                        No Pickup| 9219|
|             Overgrown Yard/Trash| 6901|
|       Front Or Side Yard Parking| 5354|
|                     Bandit Signs| 4788|
|  Aggressive Animal(Non-Critical)| 3637|
|                     Stray Animal| 3618|
|                     Damaged Cart| 3552|
|            Cart Exchange Request| 3365|
|Right Of Way/Sidewalk Obstruction| 3136|
| Junk Vehicle On Private Property| 2915|
+---------------------------------+-----+
only showing top 10 rows



### 10. What are the top 10 service request types in terms of average days late?


In [46]:
df.groupby("service_request_type").agg(F.round(F.mean("num_days_late"), 1).alias("avg_late"))\
.sort(F.desc("avg_late")).show(10, 50)

+----------------------------------------+--------+
|                    service_request_type|avg_late|
+----------------------------------------+--------+
|        Request for Research/Information|     NaN|
|                      Zoning: Junk Yards|   178.3|
|Emergency: Main and Accessory Structures|   173.5|
|          Status: On-Going CIMS Projects|   162.1|
|                    Zoning: Recycle Yard|   152.2|
|                               Complaint|   136.8|
|         Markings Installation SMO (NEW)|   111.4|
|                                 Vendors|    73.7|
|          Donation Container Enforcement|    61.6|
|                       No Address Posted|    61.1|
+----------------------------------------+--------+
only showing top 10 rows



### 11. Does number of days late depend on department? (Answer without the use of a stats test)


In [47]:
df.groupby("department").agg(F.round(F.mean("num_days_late"), 1).alias("avg_late"))\
.sort(F.desc("avg_late")).show(10, 50)

+------------------------+--------+
|              department|avg_late|
+------------------------+--------+
|            City Council|     NaN|
|        Customer Service|   112.0|
|             Solid Waste|    -1.9|
|      Parks & Recreation|    -4.5|
|            Metro Health|    -5.7|
|Trans & Cap Improvements|   -19.1|
|    DSD/Code Enforcement|   -33.8|
|    Animal Care Services|  -224.7|
+------------------------+--------+



### 12. How do number of days late depend on department and request type?


In [48]:
df.groupby("department", "service_request_type")\
.agg(F.round(F.avg("num_days_late"), 1).alias("avg_days_late")).sort("department").show(50,50)

+--------------------+------------------------------------+-------------+
|          department|                service_request_type|avg_days_late|
+--------------------+------------------------------------+-------------+
|Animal Care Services|            Injured Animal(Critical)|         -0.1|
|Animal Care Services|         Aggressive Animal(Critical)|         15.4|
|Animal Care Services|               Animal Bite(Critical)|          0.0|
|Animal Care Services|             Trapped/Confined Animal|          0.3|
|Animal Care Services|         City Council Animal Request|          1.3|
|Animal Care Services|                        Stray Animal|       -998.8|
|Animal Care Services|            Animal Cruelty(Critical)|          0.0|
|Animal Care Services|         Public Nuisance(Own Animal)|         -3.6|
|Animal Care Services|                      Animal Neglect|         15.4|
|Animal Care Services|                     Officer Standby|         -0.1|
|Animal Care Services|              An

23/10/30 04:08:45 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1066976 ms exceeds timeout 120000 ms
23/10/30 04:08:45 WARN SparkContext: Killing executors is not supported by current scheduler.
23/10/30 04:08:49 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$